In [ ]:
%cd ..

In [ ]:
import ibkr
from analysis import liveValuesForPositions
from decimal import Decimal
from ib_insync import IB
from model import Cash, Currency, Stock
import pandas as pd

In [ ]:
ib = IB()
ib.connect('127.0.0.1', port = 4001)
positions = ibkr.downloadPositions(ib, lenient=False)

# TODO: Support multi-currency portfolios
currency = Currency.USD

In [ ]:
stockPositions = [p for p in positions if isinstance(p.instrument, Stock) and p.instrument.currency == currency]
stockPositions

In [ ]:
values = liveValuesForPositions(stockPositions, ibkr.IBDataProvider(ib))

In [ ]:
portfolioValue = sum((x for x in values.values()), Cash(currency = currency, quantity = Decimal(0)))
portfolioValue

In [ ]:
rows = {p.instrument.symbol: values[p] for p in stockPositions}
pd.DataFrame.from_dict(data=rows, orient='index', columns=['Market value'])